### COCO Dataset :  train , validation and test data dumping

In [1]:
import json
import pickle
import glob
import pickle
import pandas as pd
root='D:/COCO/'
#f=open(,'r')
groups=['train','val','test']
files=[root+'annot_2017/instances_train2017.json',
       root+'annot_2017/instances_val2017.json',
       root+'annot_2017/image_info_test-dev2017.json']

img_files=[root+'train2017/train2017/*',
           root+'val2017/val2017/*',
           root+'test2017/test2017/*']

annot_dicts=[]
for file in files:
    f=open(file,'r')
    annot_dict=json.load(f)
    annot_dicts.append(annot_dict)

In [2]:
_road=['person','bicycle','car','motorcycle','airplane','bus','train','truck','boat','traffic light','stop sign']
#_home=['chair', 'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors']#, 'teddy bear', 'hair drier', 'toothbrush']

In [3]:
LABELS_ALL={}
for cat in annot_dicts[0]['categories']:
    LABELS_ALL[cat['id']]=cat['name']
LABELS={}
for cat in annot_dicts[0]['categories']:
    if cat['name'] not in _road:
        continue
    LABELS[cat['id']]=cat['name']

In [4]:
f=open(root+'_labels_coco_road','wb')
pickle.dump(LABELS,f)
f.close()

In [5]:

def save_annot(annot_dict,img_file,group):
    obj_count=pd.Series([0]*len(_road),index=_road)
    images=annot_dict['images']
    images_dict={}
    if group !='test': 
        annotations=annot_dict['annotations']
        for image in images:
            images_dict[image['id']]=[image['file_name'],image['width'],image['height']]
        annots_dict={}
        for annot in annotations:
            annots_dict[annot['image_id']]=[]
        for annot in annotations:
            annots_dict[annot['image_id']].append(annot)
        train_images={}
        for _id in annots_dict.keys():
            train_img={}
            image=images_dict[_id]
            annots=annots_dict[_id]
            train_img['id']=_id
            train_img['filename']=image[0]
            train_img['width']=image[1]
            train_img['height']=image[2]
            _objs=[]
            wf=1/image[1]
            hf=1/image[2]
            _obj_count=obj_count.copy()
            for annot in annots:
                if annot['iscrowd']==0:
                   obj={}
                   bbox=annot['bbox']
                   if LABELS_ALL[annot['category_id']] not in _road:
                         continue
                   
                   obj['name']=LABELS[annot['category_id']]
                   obj['xmin']=bbox[0]*wf
                   obj['ymin']=bbox[1]*hf
                   obj['xmax']=(bbox[0]+bbox[2])*wf
                   obj['ymax']=(bbox[1]+bbox[3])*hf
                   #if _obj_count[obj['name']] >= 10000:
                   #      _objs=[]
                   #      break
                   _objs.append(obj)
                   _obj_count[obj['name']]+=1
            objs=[]
            #for obj in _objs:
            #    if _obj_count[obj['name']] <= 5000:
            #        objs=_objs
            #        obj_count=_obj_count
            objs=_objs
            obj_count=_obj_count
            if len(objs)==0:
                continue
            train_img['object']=objs
            train_images[image[0]]=train_img
        fnames=[]  
        _data=[] 
        for i,file in enumerate(glob.glob(img_file)):
                file_name=file.split('\\')[-1]
                try:data=train_images[file_name]
                except:continue
                _data.append(data)
    if group=='test':
        _data=[]
        for i,data in enumerate(images):
            data['filename']=data['file_name']
            _data.append(data)
    print('saving files','len ',len(_data),'fname ',root+group+'_coco_dfs')
    f=open(root+group+'_coco_dfs_road','wb')
    pickle.dump(_data,f)
    f.close()
    return obj_count

In [6]:
for annot_dict,img_file,group in zip(annot_dicts[:-1],img_files[:-1],groups[:-1]):
                   obj_count=save_annot(annot_dict,img_file,group)
                   print(obj_count)

saving files len  75392 fname  D:/COCO/train_coco_dfs
person           257253
bicycle            7056
car               43533
motorcycle         8654
airplane           5129
bus                6061
train              4570
truck              9970
boat              10576
traffic light     12842
stop sign          1983
dtype: int64
saving files len  3176 fname  D:/COCO/val_coco_dfs
person           10777
bicycle            314
car               1918
motorcycle         367
airplane           143
bus                283
train              190
truck              414
boat               424
traffic light      634
stop sign           75
dtype: int64


In [7]:
obj_count

person           10777
bicycle            314
car               1918
motorcycle         367
airplane           143
bus                283
train              190
truck              414
boat               424
traffic light      634
stop sign           75
dtype: int64

### Extra functions

In [8]:
import csv

def read_data(dataset):
  f=open(dataset+'/label.csv')
  file=csv.reader(f,delimiter=',')
  data=[]
  i=0
  sc=416.0
  for line in file:
      dt=line
      H=dt[0]
      W=dt[1]
      xmin=(float(dt[2]))/sc
      ymin=(float(dt[3]))/sc
      xmax=(float(dt[4]))/sc
      ymax=(float(dt[5]))/sc

      output={
          'filename':dataset+'/images/'+str(i)+'.jpg',
          'height':H,
          'width':W,
          'object':[{'name':'None',
          'xmin':xmin*IMAGE_W,
          'ymin':ymin*IMAGE_H,
          'xmax':xmax*IMAGE_W,
          'ymax':ymax*IMAGE_H}]
          }

      data.append(output)
      i=i+1
  return data
